In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math as m
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('../input/bike-sharing-dataset/hour.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df["DateTime"]=df[['dteday','hr']].apply(lambda x:x['dteday']+" "+str(x["hr"])+":00",axis=1)

In [ ]:
def zero_pad_hour(s):
    if len(str(s))==2:
        return str(s)
    else:
        return "0"+str(s)
df["DateTime"]=df[['dteday','hr']].apply(lambda x:x['dteday']+\
                                         " "+zero_pad_hour(x["hr"])+":00",axis=1)
df["DateTime"]=pd.to_datetime(df.DateTime,format="%Y-%m-%d %H:%M")    

In [ ]:
count=df.cnt
count.index=df.DateTime
plt.figure(figsize=(12,5))
plt.plot(count[:24*7])

In [ ]:
df.weathersit=df.weathersit.astype(str)
sns.barplot(x='weathersit',y='cnt',data=df)

In [ ]:
sns.barplot(x='weathersit',y='casual',data=df)

In [ ]:
sns.scatterplot(df.casual,df.registered)

In [ ]:
sns.barplot(df.weekday,df.casual)

In [ ]:
sns.barplot(df.weekday,df.registered)

In [ ]:
sns.barplot(df.hr,df.registered)
sns.barplot(df.hr,df.casual)

In [ ]:
sns.barplot(df.mnth,df.registered)
sns.barplot(df.mnth,df.casual)

In [ ]:
plt.figure(figsize=(12,8))
sns.barplot(x='hr',y='cnt',hue='weathersit',data=df)

In [ ]:
from scipy.stats import pearsonr
print(pearsonr(df.temp,df.atemp))
sns.lmplot(x='temp',y='atemp',data=df,hue='season',fit_reg=False)

In [ ]:
#DATA CLEANING

In [ ]:
X=df[['season', 'yr', 'mnth', 'hr', \
      'workingday','weathersit', 'temp','hum', 'windspeed']].copy()
y=df.registered

In [ ]:
X["DayNight"]=X.hr.map(lambda x: 0 if x<7 else 1)

In [ ]:
def peakhour(x):
    if x in [7,8,16,17,18,19]:
        return 1
    else:
        return 0
X["PeakHours"]=X.hr.map(lambda x: peakhour(x))

In [ ]:
#lets create some dummies
dummy=pd.get_dummies(X.weathersit,prefix="WeatherSit")
#join the dummy to dataframe
X=X.join(dummy)
#drop weather sit col
X=X.drop(['weathersit'],axis=1)

In [ ]:
#Modelling
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X,y)
y_hat=lr.predict(X)
print("R sq",r2_score(y,y_hat))
print("MAE",mean_absolute_error(y,y_hat))
print("MSE",mean_squared_error(y,y_hat))
print("RMSE",mean_squared_error(y,y_hat)**0.5)

In [ ]:
#sum of  residulas should  be 0
residuals=y-y_hat
sum(residuals) 


In [ ]:
#residulas plot should be normal
sns.distplot(residuals) 

In [ ]:
#graph should follow homoskdacity, here it is hetroskadacity
plt.scatter(y_hat,residuals)
plt.hlines(y=0,xmin=-100,xmax=500,colors='r')

In [ ]:
#lets check our target variabledistribution
print(df['registered'].skew())#skewness is more , we can log transform this column
sns.distplot(df['registered'])


In [ ]:
#log Normal
log_y=np.log1p(y)
model=LinearRegression()
model.fit(X,log_y)
model.score(X,log_y)
print("R sq",r2_score(y,y_hat))
print("MAE",mean_absolute_error(y,y_hat))
print("MSE",mean_squared_error(y,y_hat))
print("RMSE",mean_squared_error(y,y_hat)**0.5)

In [ ]:
#Non Linear Model
from sklearn.preprocessing import PolynomialFeatures
poly=PolynomialFeatures(degree=4,include_bias=False)
X_poly=poly.fit_transform(X)
model=LinearRegression(n_jobs=-1)
model.fit(X_poly,log_y)
model.score(X_poly,log_y)
y_hat=np.exp(model.predict(X_poly))
r2_score(y,y_hat)


In [ ]:
residuals=y-y_hat
plt.scatter(y_hat,residuals)


In [ ]:
plt.scatter(y,y_hat)

In [ ]:
plt.plot(y[:150],color='b')
plt.plot(y_hat[:150],color='r')